# Multi node Distributed training with PyTorch

## Multi Node Application with PyTorch Distributed: Message Exchanging Example

Make sure you start a TAP session with 2 Nodes and 2 Tasks and you are <b>NOT</b> using the kernel <code>cnn_course_container</code>.

The two cells below help display Python and Bash scripts as HTML.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
from pathlib import Path
from IPython import display

In [3]:
display.Code("cnn_part5/message_passing.py")

import torch.distributed as dist
import os
import torch

LOCAL_RANK = int(os.environ['LOCAL_RANK'])
WORLD_RANK = int(os.environ['RANK'])
WORLD_SIZE = int(os.environ['WORLD_SIZE'])

def run():
    tensor = torch.zeros(1)
    
    # Send tensor to GPU device
    device = torch.device("cuda:{}".format(LOCAL_RANK))
    tensor = tensor.to(device)

    if WORLD_RANK == 0:
        for rank_recv in range(1, WORLD_SIZE):
            dist.send(tensor=tensor, dst=rank_recv)
            print('worker_{} sent data to Rank {}\n'.format(0, rank_recv))
    else:
        dist.recv(tensor=tensor, src=0)
        print('worker_{} has received data from rank {}\n'.format(WORLD_RANK, 0))

def init_processes():
     dist.init_process_group(backend="nccl", #"nccl" for using GPUs, "gloo" for using CPUs
                          world_size=WORLD_SIZE, 
                          rank=WORLD_RANK)
     run()

if __name__ == "__main__":
    init_processes()

In [4]:
display.Code("cnn_part5/message_passing.sh")

#!/bin/bash
LOCAL_RANK=$PMI_RANK

NODEFILE="$SCRATCH/hostfile"
scontrol show hostnames > $NODEFILE
if [[ -z "$NODEFILE" ]]; then
    RANKS=$NODEFILE
    NNODES=1
else
    MAIN_RANK=$(head -n 1 $NODEFILE)
    RANKS=$(tr '\n' ' ' < $NODEFILE)
    NNODES=$(< $NODEFILE wc -l)
fi


PRELOAD="/opt/apps/tacc-apptainer/1.3.3/bin/apptainer exec --nv --bind /run/user:/run/user /scratch1/07980/sli4/containers/cnn_course_latest.sif "
CMD="python3 -m torch.distributed.run --nproc_per_node 4 --nnodes $NNODES --node_rank=$LOCAL_RANK --master_addr=$MAIN_RANK --master_port=1234 cnn_part5/message_passing.py"

FULL_CMD="$PRELOAD $CMD"
echo "Training command: $FULL_CMD"

eval $FULL_CMD

In [ ]:
! chmod +x cnn_part5/message_passing.sh
! mpirun -np 2 -ppn 1 cnn_part5/message_passing.sh

## Multi Node Application with PyTorch Distributed: Damage Level Example

Download the dataset to $SCRATCH

In [5]:
display.Code("cnn_part5/copy_data.sh")

SOURCE_TAR=$1
DEST_DIR=$2

mkdir -p $DEST_DIR

FULL_CMD="cp -r $SOURCE_TAR $DEST_DIR; "
FULL_CMD+="tar zxf $DEST_DIR/data.tar.gz -C $DEST_DIR; "
FULL_CMD+="ls $DEST_DIR/Dataset_2; "
FULL_CMD+="rm $DEST_DIR/data.tar.gz; "


NODEFILE="$SCRATCH/hostfile"
scontrol show hostnames $SLURM_NODELIST > $NODEFILE

if [[ -z "${NODEFILE}" ]]; then
    RANKS=$HOSTNAME
else
    RANKS=$(tr '\n' ' ' < $NODEFILE)
fi

echo "Command: $FULL_CMD"

# Launch execute the command on each worker (use ssh for remote nodes)
RANK=0
for NODE in $RANKS; do
    if [[ "$NODE" == "$HOSTNAME" ]]; then
        echo "Launching rank $RANK on local node $NODE"
        eval $FULL_CMD &
    fi
    RANK=$((RANK+1))
done

wait

In [ ]:
! chmod +x cnn_part5/copy_data.sh 
! cnn_part5/copy_data.sh /scratch1/07980/sli4/training/cnn_course/data/data.tar.gz $SCRATCH

In [6]:
display.Code("cnn_part5/torch_train_distributed.py")

# NOTE: This is the main script of using Distributed Data Parallel to train ResNet

import sys
import os
import numpy as np
import gc

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
from datetime import datetime
import warnings
import shutil

warnings.filterwarnings("ignore",
    message="torch.distributed._all_gather_base is a private function and will be deprecated. Please use torch.distributed.all_gather_into_tensor instead.",
)

# Define the GPUs that will be used in this script
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(
    str(x) for x in list(range(torch.cuda.device_count()))
)

# Apply transformations to our data.
# The datasets transformations are the same as the ones from part 2 of this tutorial.
def load_datasets(train_path, val_path, test_path):
    val_img_transform = transforms.Compose(
        [transforms.Resize((244, 244)), transforms.ToTensor()]
    )
    train_img_transform = transforms.Compose(
        [transforms.AutoAugment(), transforms.Resize((244, 244)), transforms.ToTensor()]
    )
    train_dataset = datasets.ImageFolder(train_path, transform=train_img_transform)
    val_dataset = datasets.ImageFolder(val_path, transform=val_img_transform)
    test_dataset = (
        datasets.ImageFolder(test_path, transform=val_img_transform)
        if test_path is not None
        else None
    )

    rank = int(os.environ["RANK"])
    if rank == 0:
        print(
            f"Train set size: {len(train_dataset)}, Validation set size: {len(val_dataset)}"
        )
    return train_dataset, val_dataset, test_dataset


# Construct Dataloaders
# The DistributedSampler we use here restricts data loading to a subset of the dataset.
# In conjunction with DistributedDataParallel (shows up later in this tutorial), each process can pass a DistributedSampler instance as a DataLoader sampler, and load a subset of the original dataset that is exclusive to it.
def construct_dataloaders(train_set, val_set, test_set, batch_size, shuffle=True):
    train_sampler = DistributedSampler(dataset=train_set, shuffle=shuffle)
    val_sampler = DistributedSampler(dataset=val_set, shuffle=False)
    test_sampler = (
        DistributedSampler(dataset=test_set, shuffle=False)
        if test_set is not None
        else None
    )

    train_dataloader = torch.utils.data.DataLoader(
        train_set,
        batch_size=batch_size,
        sampler=train_sampler,
        num_workers=4,
        pin_memory=True,
    )
    val_dataloader = torch.utils.data.DataLoader(
        val_set, batch_size=batch_size, sampler=val_sampler, num_workers=4
    )
    test_dataloader = (
        torch.utils.data.DataLoader(
            test_set, batch_size, sampler=test_sampler, num_workers=4
        )
        if test_set is not None
        else None
    )

    return train_dataloader, val_dataloader, test_dataloader


# Building the Neural Network
# This is the same from part 2 of this tutorial
def getResNet():
    resnet = models.resnet34(weights="IMAGENET1K_V1")

    # Fix the conv layers parameters
    for conv_param in resnet.parameters():
        conv_param.require_grad = False

    # get the input dimension for this layer
    num_ftrs = resnet.fc.in_features

    # build the new final mlp layers of network
    fc = nn.Sequential(nn.Linear(num_ftrs, num_ftrs), nn.ReLU(), nn.Linear(num_ftrs, 3))

    # replace final fully connected layer
    resnet.fc = fc
    return resnet


# Model evaluation.
# This is implemented in the same way as part 2 of this tutorial.
@torch.no_grad()
def eval_model(data_loader, model, loss_fn, DEVICE):
    model.train(False)
    model.eval()
    loss, accuracy = 0.0, 0.0
    n = len(data_loader)

    local_rank = int(os.environ["LOCAL_RANK"])

    for i, data in enumerate(data_loader):
        x, y = data
        x, y = x.to(DEVICE), y.to(DEVICE)
        pred = model(x)
   

In [7]:
display.Code("cnn_part5/run_distributed.sh")

#!/bin/bash
LOCAL_RANK=$PMI_RANK

NODEFILE="$SCRATCH/hostfile"
scontrol show hostnames > $NODEFILE
if [[ -z "$NODEFILE" ]]; then
    RANKS=$NODEFILE
    NNODES=1
else
    MAIN_RANK=$(head -n 1 $NODEFILE)
    RANKS=$(tr '\n' ' ' < $NODEFILE)
    NNODES=$(< $NODEFILE wc -l)
fi


PRELOAD="/opt/apps/tacc-apptainer/1.3.3/bin/apptainer exec --nv --bind /run/user:/run/user /scratch1/07980/sli4/containers/cnn_course_latest.sif "
CMD="python3 -m torch.distributed.run --nproc_per_node 4 --nnodes $NNODES --node_rank=$LOCAL_RANK --master_addr=$MAIN_RANK --master_port=1234 cnn_part5/torch_train_distributed.py $@"

FULL_CMD="$PRELOAD $CMD"
echo "Training command: $FULL_CMD"

eval $FULL_CMD

In [ ]:
! chmod +x cnn_part5/run_distributed.sh
! mpirun -np 2 -ppn 1 cnn_part5/run_distributed.sh 